# Sentiment Analysis of Amazon Reviews

In this notebook we will be doing some sentiment analysis in python using two different techniques:
1. VADER (Valence Aware Dictionary and sEntiment Reasoner) - Bag of words approach
2. Roberta Pretrained Model from 🤗

As a bonus, we show how the analysis can be moved to a HuggingFace pipeline.

## Step 0. Read in Data and NLTK Basics

Note that we also need to choose a lexicon of words for the bag of words model. Usually it is better to chooise a lexicon of words that are specific to the domain, but here we use a generic lexicon. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk
# The following downloads need to be performed just once.
# They can be commented out afterwards.
#nltk.downloader.download('vader_lexicon')
#nltk.download('averaged_perceptron_tagger_eng')
#nltk.download('maxent_ne_chunker_tab')

We read in the data, which comes from Amazon reviews of fine/up-market food products. We have selected the first 999 such reviews for convenience.

In [ ]:
df = pd.read_csv('data/Reviews1000.csv')
print(df.shape)
df.head()

### EDA

We now perform some limited EDA on the reviews. We are interested in the number of reviews by starr rating (1 to 5 stars).

In [ ]:
ax = df['Score'].value_counts().sort_index() \
    .plot(kind='bar',
          title='Count of Reviews by Stars',
          figsize=(10, 5))
ax.set_xlabel('Review Stars')
plt.show()

## Text processing using NLTK

In [ ]:
example = df['Text'][100]
print(example)

In [ ]:
tokens = nltk.word_tokenize(example)
tokens[:10]

In [ ]:
tagged = nltk.pos_tag(tokens)
tagged[:10]

In [ ]:
entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

## Step 1. VADER Sentiment Scoring

We use NLTK's `SentimentIntensityAnalyzer` to get the neg/neu/pos scores of the text using VADER.

- This uses a "bag of words" approach:
    1. Stop words are removed
    2. each word is scored and combined to a total score.
 
The `SentimentInstensityAnalyzer` uses the bag of words from the standard lexicon (not specific to food reviews).

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

We give it what might be considered a positive statement, but VADER decides it is only neutral. That is because the meaning of "passed" (in the sense of passing an exam) is missed.

In [ ]:
stmt = 'The exam results are in, and she passed.'
sia.polarity_scores(stmt)

Even when the sentiment is more clearly negative, VADER is agaiun confused, probably because the statement is about Trump but is not clearly the writer's belief.

In [ ]:
stmt = 'Trump was the worst President in recent history.'
sia.polarity_scores(stmt)

The food review  regarding the apples is somewhat unclear - it has a positive (but indirect) Shakespearean quotation and comment implying dissatisfaction with the price. Deciding the review is neutral is probably the right interpretation here.

In [ ]:
sia.polarity_scores(example)

Now we can run the sentiment analyser over the entire dataset, adding the estimated sentiment as a new dict.

Note the use of `tqdm` to provide a progress bar as the code works through the dataframe.

In [ ]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Text']
    myid = row['Id']
    res[myid] = sia.polarity_scores(text)

Now we add the sentiment score array to the data, as an additional column, saving the combination in a new dataframe called `vaders`.

In [ ]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'Id'})
vaders = vaders.merge(df, how='left')
vaders.head()

## Plot VADER results

In [ ]:
ax = sns.barplot(data=vaders, x='Score', y='compound')
ax.set_title('Compound Score by Amazon Star Review')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 3))
sns.barplot(data=vaders, x='Score', y='pos', ax=axs[0])
sns.barplot(data=vaders, x='Score', y='neu', ax=axs[1])
sns.barplot(data=vaders, x='Score', y='neg', ax=axs[2])
axs[0].set_title('Positive')
axs[1].set_title('Neutral')
axs[2].set_title('Negative')
plt.tight_layout()
plt.show()

# Step 3. Roberta Pretrained Model

- Use a model trained of a large corpus of data.
- Transformer model accounts for the words but also the context related to other words.

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
# VADER results on example
print(example)
sia.polarity_scores(example)

In [ ]:
# Run for Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['Text']
        myid = row['Id']
        vader_result = sia.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = polarity_scores_roberta(text)
        both = {**vader_result_rename, **roberta_result}
        res[myid] = both
    except RuntimeError:
        print(f'Broke for id {myid}')

In [ ]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'Id'})
results_df = results_df.merge(df, how='left')
results_df.columns

## Compare Scores between models

# Step 3. Combine and compare

In [ ]:
sns.pairplot(data=results_df,
             vars=['vader_neg', 'vader_neu', 'vader_pos',
                  'roberta_neg', 'roberta_neu', 'roberta_pos'],
            hue='Score',
            palette='tab10')
plt.show()

# Step 4: Review Examples:

- Positive 1-Star and Negative 5-Star Reviews

Lets look at some examples where the model scoring and review score differ the most.

In [ ]:
results_df.query('Score == 1') \
    .sort_values('roberta_pos', ascending=False)['Text'].values[0]

In [ ]:
results_df.query('Score == 1') \
    .sort_values('vader_pos', ascending=False)['Text'].values[0]

Negative sentiment 5-Star view

In [ ]:
results_df.query('Score == 5') \
    .sort_values('roberta_neg', ascending=False)['Text'].values[0]

In [ ]:
results_df.query('Score == 5') \
    .sort_values('vader_neg', ascending=False)['Text'].values[0]

# Extra: The Transformers Pipeline
- Quick & easy way to run sentiment predictions

In [ ]:
from transformers import pipeline
pipelineType = "sentiment-analysis"
sent_pipeline = pipeline(pipelineType)

In [ ]:
sent_pipeline('I love good weather')

In [ ]:
sent_pipeline('The hotel is not too bad.')

In [ ]:
sent_pipeline('The song is terrible.')

# The End